In [1]:
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

from datetime import datetime, timedelta, date, time
import dateutil
from dateutil.parser import parse
import calendar
import numpy as np
from circle_cal.model import CalendarElement
import circle_cal as cc
import circle_cal.model as model
import circle_cal.plot as ccplot
import pathlib
from rich import print
import pytz
from pytz import timezone
import workalendar

In [2]:
cred = pathlib.Path('/Users/kdavis10/.config/.credentials')
cred_json = "pygooglecal.apps.googleusercontent.com.json"

try:
    gcal = GoogleCalendar(credentials_path=cred / cred_json)
except Exception as e:
    (cred / "token.pickle").unlink()
    print("Token expired, re-run this cell.")

In [3]:
year = y24 = model.Year(2024)

traces = []

clist = list(gcal.get_calendar_list())
sel_cal_summ = ["http://universerevealed.nd.edu/?eme_ical=public",
                "NYT Astronomy and Space Calendar",
                "kdavis10@nd.edu",
                "ND-DVT Calendar",
                "CoS College-wide Events & Conferences",
                "Keith Davis",
                "Axis Ludi Calendar",
                "Holidays in United States",
                "Academic Calendar",
                "Seasons"]

selcal = [c for c in clist if c.summary in sel_cal_summ]
print(selcal)

ws = year.weekends()

[
    <CalendarListEntry None - (http://universerevealed.nd.edu/?eme_ical=public)>,
    <CalendarListEntry None - (NYT Astronomy and Space Calendar)>,
    <CalendarListEntry None - (kdavis10@nd.edu)>,
    <CalendarListEntry None - (ND-DVT Calendar)>,
    <CalendarListEntry None - (CoS College-wide Events & Conferences)>,
    <CalendarListEntry None - (Keith Davis)>,
    <CalendarListEntry None - (Axis Ludi Calendar)>,
    <CalendarListEntry ND Academic Calendar - (Academic Calendar)>,
    <CalendarListEntry None - (Holidays in United States)>,
    <CalendarListEntry None - (Seasons)>
]

In [4]:
df = ccplot.selected_cals_to_dataframe(gcal, selcal, y24)
df

AttributeError: 'Year' object has no attribute 'end'

In [ ]:
ii = pd.IntervalIndex([pd.Interval(pd.Timedelta(days=0), pd.Timedelta(hours=23, minutes=59, seconds=59)),
                       pd.Interval(pd.Timedelta(hours=23, minutes=59, seconds=59), pd.Timedelta(days=1)),
                       pd.Interval(pd.Timedelta(days=1), pd.Timedelta(days=31)),
                       pd.Interval(pd.Timedelta(days=31), pd.Timedelta(days=10000))])
intervalsdf = pd.DataFrame(data=["hours", "day", "weeks", "months"], index=ii, columns=["name"])
intervalsdf["opacity"] = [1, 0.8, 0.7, 0.5]
intervalsdf["rbase"] = list(reversed([0.5, 0.6, 0.7, 0.8]))



In [7]:
df["scale"] = pd.cut(df["duration"], intervalsdf.index)

In [8]:
df["ring"] = intervalsdf["name"].loc[df["scale"]].reset_index(drop=True)
df["opacity"] = intervalsdf["opacity"].loc[df["scale"]].reset_index(drop=True)
df["rbase"] = intervalsdf["rbase"].loc[df["scale"]].reset_index(drop=True)

In [9]:
def calendar_ticks(year):
    tickvals = []
    ticktext = []
    for m in year:
        tickvals.append(year.to_theta(m.start))
        #tickvals.append(ts_to_theta(model.to_timestamp(m.mid),
         #                           model.to_timestamp(year.start.datetime()),
          #                          (year.duration / timedelta(days=1)) / 360))
        #ticktext.append("")
        ticktext.append(m.name)
    return (tickvals, ticktext)
        

##def ring_to_base(ring, intervalsdf):
 #   dr = 1/(len(intervalsdf) + 1)
 #   rvalues = [(r+1) * dr for r in range(len(intervalsdf))]   
 #   d = dict(zip(intervalsdf.name.values, reversed(rvalues)))
 #   return d[ring]
    
year=y24

df["midtheta"] = df.mid.apply(year.to_theta)
df["width"] = df.duration.apply(year.to_theta)
#df["rbase"] = df.ring.apply(lambda x: ring_to_base(x, intervalsdf))
#df["rbase"] = .5
df["r"] = .1 * .97 #1/(len(intervalsdf) + 1) * .97


In [10]:
dring = df
try:
    del f
except NameError:
    pass
f = subplotsfig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'polar'}]])
f = go.Figure(f)
#f = go.Figure()

for cal in pd.unique(df.calendar):
    df = dring[dring["calendar"] == cal]
    f.add_trace(go.Barpolar(theta=df.midtheta, r=df.r, width=df.width, base=df.rbase,
                            text=df.summary, marker_color=df.color, name=cal, #marker_opacity=df.opacity
                            customdata=df[["start", "end"]],
                            
                           )
               )
df = dring
tickvals, ticktext = calendar_ticks(y24)

f.add_trace(go.Barpolar(theta=[o_theta(model.to_timestamp(w.mid), model.to_timestamp(y24.start.datetime())) for w in ws],
                        #r=len(ws) * [.167], base=.167,
                        r=len(ws) * [.1], base=.4,
                        marker_color="#E1E1E1", marker_opacity=.5,
                        width=[(w.duration / year.duration * 360) for w in ws], name="Weekends",
                       customdata=[(w.start, w.end) for w in ws])
           )

f.update_layout(height=800)
f.update_layout(
    template=None,
    polar = dict(
        radialaxis = dict(tickvals=[intervalsdf.rbase],
                          showticklabels=False, ticks='',
                         ),
        angularaxis = dict(
            rotation = -90,
            showticklabels=True,
            ticks='outside',
            ticktext=ticktext,
            tickvals=tickvals,
        )
    )
)
f.update_traces(hovertemplate="%{text}<br>Start: %{customdata[0]|%Y/%m/%d %H:%M:%S.%L}"
               "<br>End: %{customdata[1]|%Y/%m/%d %H:%M:%S.%L}")
#f.add_trace(go.Sunburst(yd))
f.add_trace(go.Scatterpolar(r=[0, 1], theta=2*[ts_to_theta(model.to_timestamp(datetime.now()),
                                                           model.to_timestamp(y24.start.datetime()),
                                                          366 / 360)], name="Now", marker_color="black", text=["Today"]*2))

for t in f.data:
    f.add_trace(t, 1, 2)

f.update_layout(polar2=dict(angularaxis=f.layout["polar"]["angularaxis"],
                           radialaxis=f.layout["polar"]["radialaxis"]))
f.update_layout(polar2=dict(sector=[90,100]))

f.show()

AttributeError: 'Year' object has no attribute 'is_whole_months'

In [12]:
f.layout["polar"]

layout.Polar({
    'angularaxis': {'rotation': -90,
                    'showticklabels': True,
                    'ticks': 'outside',
                    'ticktext': [January, February, March, April, May, June, July,
                                 August, September, October, November, December],
                    'tickvals': [0.0, 30.491803278688526, 59.016393442622956,
                                 89.4672131147541, 118.97540983606558,
                                 149.46721311475412, 178.97540983606558,
                                 209.46721311475412, 239.95901639344265,
                                 269.4672131147541, 299.9590163934426,
                                 329.5081967213115]},
    'domain': {'x': [0.0, 0.45], 'y': [0.0, 1.0]},
    'radialaxis': {'showticklabels': False, 'ticks': '', 'tickvals': [[0.8, 0.7, 0.6, 0.5]]}
})

In [13]:
g = iter(year)

In [14]:
for m in g:
    print(m)

{'year': 2024, 'month': 1, 'type': 'CalendarElement'}

{'year': 2024, 'month': 2, 'type': 'CalendarElement'}

{'year': 2024, 'month': 3, 'type': 'CalendarElement'}

{'year': 2024, 'month': 4, 'type': 'CalendarElement'}

{'year': 2024, 'month': 5, 'type': 'CalendarElement'}

...

{'year': 2024, 'month': 8, 'type': 'CalendarElement'}

{'year': 2024, 'month': 9, 'type': 'CalendarElement'}

{'year': 2024, 'month': 10, 'type': 'CalendarElement'}

{'year': 2024, 'month': 11, 'type': 'CalendarElement'}

{'year': 2024, 'month': 12, 'type': 'CalendarElement'}